<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/09.%20Chapter9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 9: Unsupervised Learning Techniques

Bab ini membahas **Unsupervised Learning**, di mana algoritma belajar dari data yang tidak memiliki label. Fokusnya adalah pada tiga tugas utama:
1.  **Clustering**: Mengelompokkan instance yang mirip ke dalam grup atau *cluster*.
2.  **Anomaly Detection**: Mendeteksi instance yang sangat menyimpang dari norma (*outlier*).
3.  **Density Estimation**: Mengestimasi fungsi kepadatan probabilitas (PDF) dari data.

* **Clustering**:
    * **K-Means**: Salah satu algoritma clustering yang paling populer. Ia bekerja dengan cara:
        1.  Menempatkan *k* buah *centroid* secara acak.
        2.  Memberi label pada setiap instance berdasarkan *centroid* terdekat.
        3.  Memperbarui posisi *centroid* dengan menghitung rata-rata dari semua instance dalam clusternya.
        4.  Mengulangi langkah 2 dan 3 hingga *centroid* tidak lagi bergerak.
        * **Kelemahan**: Harus menentukan jumlah cluster (*k*), sensitif terhadap inisialisasi, dan kesulitan menangani cluster dengan bentuk non-bola, ukuran bervariasi, atau kepadatan yang berbeda.
        * **Menemukan *k***: Teknik seperti **Elbow Method** (menggunakan metrik *inertia*) dan **Silhouette Score** dapat membantu menemukan jumlah cluster yang optimal.
    * **DBSCAN** (*Density-Based Spatial Clustering of Applications with Noise*): Algoritma yang mendefinisikan cluster sebagai wilayah padat yang berkesinambungan.
        * Sangat baik dalam menemukan cluster dengan bentuk yang arbitrer (tidak harus bola).
        * Tahan terhadap *outlier* (menganggapnya sebagai *noise*).
        * Tidak mengharuskan kita untuk menentukan jumlah cluster, tetapi memerlukan pengaturan *hyperparameter* `eps` (radius lingkungan) dan `min_samples`.

* **Gaussian Mixture Models (GMM)**:
    * Model probabilistik yang mengasumsikan data berasal dari campuran beberapa distribusi Gaussian dengan parameter yang tidak diketahui.
    * Setiap cluster dapat memiliki bentuk elips, ukuran, kepadatan, dan orientasi yang berbeda, membuatnya lebih fleksibel daripada K-Means.
    * Dilatih menggunakan algoritma **Expectation-Maximization (EM)**.
    * **Aplikasi**: Dapat digunakan untuk *density estimation*, *clustering*, dan *anomaly detection*.
    * **Memilih Jumlah Cluster**: Dapat menggunakan kriteria informasi seperti **BIC** (*Bayesian Information Criterion*) atau **AIC** (*Akaike Information Criterion*). Model **Bayesian GMM** bahkan dapat secara otomatis menemukan jumlah cluster yang optimal.

### 1. K-Means
Pertama, kita akan membuat data sintetis untuk mendemonstrasikan K-Means.

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# Membuat dataset dengan 5 blob (cluster)
blob_centers = np.array(
    [[ 0.2,  2.3], [-1.5 ,  2.3], [-2.8,  1.8], [-2.8,  2.8], [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

# Melatih model K-Means
k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)

# y_pred berisi label cluster untuk setiap instance
# kmeans.cluster_centers_ berisi lokasi centroid
print("Lokasi Centroid:\n", kmeans.cluster_centers_)

# Fungsi untuk mem-plot cluster
def plot_clusters(X, y=None, centroids=None, k=None):
    if y is not None:
        plt.scatter(X[:, 0], X[:, 1], c=y, s=1)
    else:
        plt.scatter(X[:, 0], X[:, 1], s=1)
    if centroids is not None:
        plt.scatter(centroids[:, 0], centroids[:, 1], s=50, c='red', marker='x')
    if k:
      plt.title(f"k={k}")

plt.figure(figsize=(8, 4))
plot_clusters(X, y=y_pred, centroids=kmeans.cluster_centers_, k=k)
plt.show()
```

### 2. Menemukan Jumlah Cluster Optimal (Elbow Method & Silhouette Score)
Kita tidak selalu tahu jumlah cluster yang tepat. Dua metode berikut dapat membantu.

```python
from sklearn.metrics import silhouette_score

# --- Elbow Method (menggunakan inertia) ---
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$")
plt.ylabel("Inertia")
plt.title("Elbow Method")
plt.grid(True)

# --- Silhouette Score ---
silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]] # score butuh minimal 2 cluster

plt.subplot(122)
plt.plot(range(2, 10), silhouette_scores, "bo-")
plt.xlabel("$k$")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score")
plt.grid(True)

plt.show()
```
Dari plot di atas, "siku" pada metode *elbow* terlihat jelas di k=5. Demikian pula, *silhouette score* mencapai puncaknya di k=5, mengonfirmasi bahwa 5 adalah jumlah cluster yang baik.

### 3. DBSCAN
DBSCAN sangat baik untuk cluster dengan bentuk non-bola, seperti dataset `moons`.

```python
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

# Membuat dataset moons
X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)

# Melatih model DBSCAN
# eps adalah radius lingkungan, min_samples adalah jumlah tetangga minimum
dbscan = DBSCAN(eps=0.2, min_samples=5)
dbscan.fit(X)

# Label -1 menunjukkan outlier/noise
print("Label unik dari DBSCAN:", np.unique(dbscan.labels_))

# Plot hasil
plt.figure(figsize=(8, 4))
plot_clusters(X, y=dbscan.labels_)
plt.title("Hasil DBSCAN")
plt.show()
```
DBSCAN berhasil mengidentifikasi dua cluster berbentuk bulan sabit dengan benar, sesuatu yang tidak bisa dilakukan oleh K-Means.

### 4. Gaussian Mixture Models (GMM)
GMM dapat menangani cluster berbentuk elips dengan baik.

```python
from sklearn.mixture import GaussianMixture

# Menggunakan data blob dari contoh K-Means
gmm = GaussianMixture(n_components=5, n_init=10, random_state=42)
gmm.fit(X)

# Memprediksi cluster
y_pred_gmm = gmm.predict(X)

# Plot hasil
plt.figure(figsize=(8, 4))
plot_clusters(X, y=y_pred_gmm, centroids=gmm.means_, k=5)
plt.title("Hasil Gaussian Mixture Model")
plt.show()

# Menghitung BIC dan AIC untuk memilih jumlah komponen
bics = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(X).bic(X)
        for k in range(1, 10)]
aics = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(X).aic(X)
        for k in range(1, 10)]

plt.figure(figsize=(8, 4))
plt.plot(range(1, 10), bics, "bo-", label="BIC")
plt.plot(range(1, 10), aics, "go--", label="AIC")
plt.xlabel("$k$")
plt.ylabel("Information Criterion")
plt.legend()
plt.grid(True)
plt.show()
```
Plot BIC/AIC menunjukkan bahwa nilai terendah ada di k=5, yang sesuai dengan jumlah cluster sebenarnya dalam data kita.

